In [1]:
# draw arc in 3d space
# https://www.codementor.io/hirengadhiya/python-matplotlib-plotting-an-arc-in-3d-plot-wor3d4gzg
# Callin Switzer
# 16 July 2019

In [2]:

# imports
%matplotlib qt 
# uncomment above on for interactive plots
from mpl_toolkits.mplot3d import Axes3D
import numpy as np
import matplotlib.pyplot as plt
import numpy as np
from matplotlib import interactive
interactive(True)
import math
import sys

In [19]:
# 3 pts convert to r, theta, phi
def points2sphere(p1,p2,p3):
    n = np.cross(p1,p2)+np.cross(p2,p3)+np.cross(p3,p1)
    n = n / (n**2).sum()**0.5
    
    u = np.cross((p3-p1),n)
    u = u / (u**2).sum()**0.5
    v = np.cross(n,u)
    v = v / (v**2).sum()**0.5
    R = np.vstack((u,v)).transpose()
    
    d = np.dot(n,p1)
    
    a = R.transpose()*np.matrix(p1).T
    b = R.transpose()*np.matrix(p2).T
    c = R.transpose()*np.matrix(p3).T
    
    
    
    A = np.matrix([[b[0,0]-a[0,0],b[1,0]-a[1,0]],[c[0,0]-a[0,0],c[1,0]-a[1,0]]])
    solut = np.linalg.inv((A))*np.matrix(((b[0,0]**2+b[1,0]**2-a[0,0]**2-a[1,0]**2)/2, 
                      (c[0,0]**2+c[1,0]**2-a[0,0]**2-a[1,0]**2)/2)).T
    
    Q1 = np.matrix(n*d)
    Q2 = np.matrix(R)*solut
    Q = Q1+Q2.T
    
    #print(p1-Q)
    r = np.linalg.norm(p1 - Q)
    theta = np.arccos(np.dot((p3-Q),(p1-Q).T)[0,0]/(np.linalg.norm(p1 - Q)*np.linalg.norm(p3 - Q)))
    #theta = theta[0,0]
    #print(p3-Q)
    
    phi = np.arccos(abs(n[0]))-np.pi/4
    
    return(r,theta,phi)

In [4]:
# p1 = np.array([0,0,0])
# p2 = np.array([1,0,1])
# p3 = np.array([0,1,1])

# radius1,theta1,phi1 = points2sphere(p1,p2,p3)

# print(radius,theta,phi)

In [5]:
def cart2sphere(x, y, z):
    r = np.sqrt(x**2 + y**2 + z**2)
    theta = np.arccos(z, r)
    phi = np.arctan2(y, x)
    return(r, theta, phi)

def sphere2cart(r, theta, phi):
    theta = theta - np.pi/2
    x = r * np.sin(theta)* np.cos(phi)
    y = r * np.sin(theta)* np.sin(phi)
    z = r * np.cos(theta)
    return(x, y, z)

def pol2cart(rho, phi):
    x = rho * np.cos(phi)
    y = rho * np.sin(phi)
    return(x, y)

def translate(r,theta,phi):
    c1 = np.cos(theta)
    s1 = np.sin(-theta)
    c2 = np.cos(phi)
    s2 = np.sin(phi)
#     T1 = np.matrix([[c2,-s2,0,0],[s2,c2,0,0],[0,0,1,0],[0,0,0,1]])
#     T2 = np.matrix([[1,0,0,r*(c1-1)],[0,c1,-s1,r*s1],[0,s1,c1,0],[0,0,0,1]])
#     T3 = np.matrix([[c2,s2,0,0],[-s2,c2,0,0],[0,0,1,0],[0,0,0,1]])
#     T = T1*T2*T3
    T = np.matrix([[(c2*c2)*(c1-1)+1,s2*c2*(c1-1),-c2*s1,r*c2*(c1-1)],[s2*c2*(c1-1),c2*c2*(1-c1)+c1,-s2*s1,r*s2*(c1-1)],[c2*s1,s2*s1,c1,r*s1],[0,0,0,1]])
    return(T)

In [6]:
# define values 
theta1 = np.pi/4 # arclength in radians
radius1 = 50 # raduis of circle
#k1 = 1/radius1 # if you want to use k instead of radius
phi1 = np.pi*2/5 # angle of circle in xy plane

radius2 = 20
theta2 = np.pi*3/5
phi2 = np.pi*4/4

radius3 = 30
theta3 = np.pi*2/5
phi3 = np.pi*4/4

In [7]:
# discretize for plotting
arcIndex1 = np.linspace(0, theta1, num = 100)
arcIndex2 = np.linspace(0, theta2, num = 100)
arcIndex3 = np.linspace(0, theta3, num = 100)
X1, Y1, Z1, = sphere2cart(radius1, arcIndex1, phi1)
X2, Y2, Z2, = sphere2cart(radius2, arcIndex2, phi2)
X3, Y3, Z3, = sphere2cart(radius3, arcIndex3, phi3)

In [8]:
# move center or arc to xy plane
x1, y1 = pol2cart(radius1, phi1)
X1 += x1
Y1 += y1

In [9]:
#plot rigid connector
Rx1 = [X1[-1],X1[-1]+5*np.cos(theta1-np.pi/2)*np.cos(phi1)]
Ry1 = [Y1[-1],Y1[-1]+5*np.cos(theta1-np.pi/2)*np.sin(phi1)]
Rz1 = [Z1[-1],Z1[-1]-5*np.sin(theta1-np.pi/2)]

#v_rigid = [5*np.cos(theta1-np.pi/2)*np.cos(phi1),5*np.cos(theta1-np.pi/2)*np.sin(phi1),-5*np.sin(theta1-np.pi/2)]

In [10]:
# Transform the 2nd arc
T1 = translate(radius1,theta1,phi1)
X2n = np.reshape(np.array(np.transpose((T1*np.stack((X2,Y2,Z2,np.ones((100)))))[0])),(100,))
Y2n = np.reshape(np.array(np.transpose((T1*np.stack((X2,Y2,Z2,np.ones((100)))))[1])),(100,))
Z2n = np.reshape(np.array(np.transpose((T1*np.stack((X2,Y2,Z2,np.ones((100)))))[2])),(100,))

In [11]:
x2, y2 = pol2cart(radius2, phi2)
X2n += x2
Y2n += y2
X2n +=Rx1[-1]
Y2n +=Ry1[-1]
Z2n +=Rz1[-1]

In [12]:
# translate the 2bd curve
X2n += Rx1[-1]-X2n[0]
Y2n +=Ry1[-1]-Y2n[0]
Z2n +=Rz1[-1]-Z2n[0]

In [13]:
#plot 2nd rigid connector
Rx2 = [X2n[-1],(X2n[-1]-X2n[-2])*500/theta2/radius2+X2n[-2]]
Ry2 = [Y2n[-1],(Y2n[-1]-Y2n[-2])*500/theta2/radius2+Y2n[-2]]
Rz2 = [Z2n[-1],(Z2n[-1]-Z2n[-2])*500/theta2/radius2+Z2n[-2]]

In [14]:
# Transform the 3rd arc
T2 = translate(radius2,theta2,phi2)
X3n = np.reshape(np.array(np.transpose((T1*T2*np.stack((X3,Y3,Z3,np.ones((100)))))[0])),(100,))
Y3n = np.reshape(np.array(np.transpose((T1*T2*np.stack((X3,Y3,Z3,np.ones((100)))))[1])),(100,))
Z3n = np.reshape(np.array(np.transpose((T1*T2*np.stack((X3,Y3,Z3,np.ones((100)))))[2])),(100,))

In [15]:
x3, y3 = pol2cart(radius3, phi3)
X3n += x3
Y3n += y3

In [16]:
# translate the 3rd curve
X3n += Rx2[-1]-X3n[0]
Y3n +=Ry2[-1]-Y3n[0]
Z3n +=Rz2[-1]-Z3n[0]

In [17]:
#plot 3nd rigid connector
Rx3 = [X3n[-1],(X3n[-1]-X3n[-2])*500/theta3/radius3+X3n[-2]]
Ry3 = [Y3n[-1],(Y3n[-1]-Y3n[-2])*500/theta3/radius3+Y3n[-2]]
Rz3 = [Z3n[-1],(Z3n[-1]-Z3n[-2])*500/theta3/radius3+Z3n[-2]]

In [18]:
fig = plt.figure()
ax = fig.gca(projection='3d')
# plot arc
ax.plot(X1, Y1, Z1, label='arc1')
ax.plot(Rx1,Ry1,Rz1,label='rigid1')
ax.plot(X2n, Y2n, Z2n, label='arc2')
ax.plot(Rx2,Ry2,Rz2,label='rigid2')
ax.plot(X3n,Y3n,Z3n,label='arc3')
ax.plot(Rx3,Ry3,Rz3,label='rigid2')


# plot axes
ax.plot(np.zeros(100), np.zeros(100), np.linspace(-np.max(np.abs(Z1)), np.max(np.abs(Z1)), 100), c= "black", alpha = 0.2)
ax.plot(np.zeros(100), np.linspace(-np.max(np.abs(Z1)), np.max(np.abs(Z1)), 100), np.zeros(100),  c= "black", alpha = 0.2)
ax.plot(np.linspace(-np.max(np.abs(Z1)), np.max(np.abs(Z1)), 100), np.zeros(100), np.zeros(100),  c= "black", alpha = 0.2)

# plot center of circle
#ax.scatter(np.array([x1]), np.array([y1]), np.array([0]), c = 'orange', label = "center")
# ax.scatter(p1[0], p1[1], p1[2], c = 'orange', label = "p1")
# ax.scatter(p2[0], p2[1], p2[2], c = 'orange', label = "p2")
# ax.scatter(p3[0], p3[1], p3[2], c = 'orange', label = "p3")

ax.set_xlabel('x')
ax.set_ylabel('y')
ax.set_zlabel('z')

# #plot start point
# ax.scatter(X2n[0], Y2n[0], Z2n[0], c = 'blue', label = "startpoint")
# #plot endpoint
# ax.scatter(X2n[-1], Y2n[-1], Z2n[-1], c = 'red', label = "endpoint")

#plot start point
#ax.scatter(X2n[0], Y2n[0], Z2n[0], c = 'blue', label = "startpoint")
#plot endpoint
#ax.scatter(X2n[-1], Y2n[-1], Z2n[-1], c = 'red', label = "endpoint")


# plot projection on each axis
#ax.plot(X1, np.zeros(len(X1)), np.zeros(len(X1)), color = "red", label = "X projection")
#ax.plot(np.zeros(len(X1)), Y1, np.zeros(len(X1)), color = "red", label = "Y projection")
#ax.plot(np.zeros(len(X1)), np.zeros(len(X1)), Z1, color = "red", label = "Z projection")

ax.legend()
plt.show()

<ipython-input-18-9151f295ee9f>:2: MatplotlibDeprecationWarning: Calling gca() with keyword arguments was deprecated in Matplotlib 3.4. Starting two minor releases later, gca() will take no keyword arguments. The gca() function should only be used to get the current axes, or if no axes exist, create new axes with default keyword arguments. To create a new axes with non-default arguments, use plt.axes() or plt.subplot().
  ax = fig.gca(projection='3d')
